In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

In [2]:
template = "https://domicilios.com/restaurantes/bogota?isOnline=1&channels=pago-online&paymentMethods=112&bt=RESTAURANT&page={0}"
print(template.format(1))
print(template.format(1000))

https://domicilios.com/restaurantes/bogota?isOnline=1&channels=pago-online&paymentMethods=112&bt=RESTAURANT&page=1
https://domicilios.com/restaurantes/bogota?isOnline=1&channels=pago-online&paymentMethods=112&bt=RESTAURANT&page=1000


In [4]:
rest_links = {}
driver = webdriver.Chrome(executable_path="./chromedriver")
i = 1
while True:
    driver.get(template.format(i))
    i += 1
    time.sleep(4)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    restaurantes = soup.find_all(class_="restaurantData")
    if len(restaurantes) < 10:
        break
    for res in restaurantes:
        name = res.find(class_="arrivalName").get_text()
        try:
            link = res.find("a", {"rel": "nofollow"})["href"]
        except:
            link = None
        rest_links[name] = link
driver.quit()

KeyboardInterrupt: 

In [7]:
import pickle

with open("restaurantes_links.p", "wb") as f:
    pickle.dump(rest_links, f)

In [31]:
with open("restaurantes_links.p", "rb") as f:
    all_restaurants_links = pickle.load(f)

import json
with open('ppc_suba_comentarios.json', 'rb') as f_comments:
    comments = json.load(f_comments)

In [28]:
print(comments)

[{'text': 'Parecía arroz del día anterior, trasnochado y de mal gusto', 'author': 'Raul fernando', 'previousOrders': '2', 'previousReviews': '0', 'date': '21 sep, 2019', 'worstRating': '1', 'ratingValue': '3.7', 'bestRating': '5'}, {'text': 'El pollo más rico que tiene colombia ❤?❤?', 'author': 'Andres  ramos', 'previousOrders': '25', 'previousReviews': '0', 'date': '25 jul, 2019', 'worstRating': '1', 'ratingValue': '5.0', 'bestRating': '5'}, {'text': 'redimi y cupón de $7.000 y me cobraron la cuenta completa sin el descuento del cupon', 'author': 'Camilo espitia', 'previousOrders': '266', 'previousReviews': '0', 'date': '20 jun, 2019', 'worstRating': '1', 'ratingValue': '4.0', 'bestRating': '5'}, {'text': 'Excelente servicio, el domiciliario es muy respetuoso y amable', 'author': 'Laura sanchez', 'previousOrders': '274', 'previousReviews': '0', 'date': '09 jun, 2019', 'worstRating': '1', 'ratingValue': '5.0', 'bestRating': '5'}, {'text': 'Gratamente Sorprendida, han Mejorado la calida

In [32]:
import nltk

nltk.download("stopwords")
from nltk.corpus import stopwords

def remove_stopwords(text):
    return [word for word in text if word not in stopwords.words('spanish')]

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ingenieria/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
def remove_punct(text):
    puntuacion = "! ? , . : ;".split()
    new_text = text
    for sign in puntuacion:
        new_text = new_text.replace(sign, '')
    new_text = new_text.lower()
    new_text = " ".join(remove_stopwords(new_text.split()))
    return new_text

In [37]:
for comment in comments:
    comment['text_nopunct'] = remove_punct(comment['text'])

In [38]:
with open("archivo.txt", "w") as f:
    for c in comments[:10]:
        f.write(c['text_nopunct'])
        f.write("\n")

In [39]:
import fasttext
modelo = fasttext.train_unsupervised('archivo.txt', dim=10)
modelo.get_sentence_vector('faltan cuatro minutos para la una')

array([-0.00724311, -0.01367409, -0.11141652, -0.15393221,  0.04881471,
        0.11089667,  0.09096009, -0.11503991,  0.23437262, -0.1442636 ],
      dtype=float32)

In [40]:
for comment in comments:
    comment['vector'] = modelo.get_sentence_vector(comment['text_nopunct'])